In [30]:
import re
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
from selenium import webdriver
from bson import json_util, ObjectId
from pandas.io.json import json_normalize
import json

In [2]:
path = {'executable_path': './chromedriver.exe'}
browser = Browser('chrome', **path, headless = False)
url = 'https://redplanetscience.com/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')





In [3]:
title = soup.find('div', class_ = 'content_title').get_text()
print(title)
paragraph = soup.find('div', class_ = 'article_teaser_body').get_text()
print(paragraph)

The Extraordinary Sample-Gathering System of NASA's Perseverance Mars Rover
Two astronauts collected Moon rocks on Apollo 11. It will take three robotic systems working together to gather up the first Mars rock samples for return to Earth.


In [4]:

path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **path, headless = False)
url_image = 'https://spaceimages-mars.com/'
browser.visit(url)

html = browser.html
soup = bs(html, 'html.parser')

article = soup.find('img', class_='headerimage fade-in')
print(article)

full_path = 'https://spaceimages-mars.com/image/featured/mars2.jpg'
full_path




None


'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [33]:
url_facts = 'https://galaxyfacts-mars.com/'
df_facts = pd.read_html(url_facts, header=0)[0]
df_facts
df_facts.set_index('Mars - Earth Comparison')

df_facts



,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-87 to -5 °C,-88 to 58°C


In [34]:
def mongo_to_dataframe(mongo_data):

    sanitized = json.loads(json_util.dumps(mongo_data))
    normalized = json_normalize(sanitized)
    df = pd.DataFrame(normalized)

    return df

mongo_to_dataframe(df_facts) 

<ipython-input-34-b38d247f94c2>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  normalized = json_normalize(sanitized)


,Mars - Earth Comparison.0,Mars - Earth Comparison.1,Mars - Earth Comparison.2,Mars - Earth Comparison.3,Mars - Earth Comparison.4,Mars - Earth Comparison.5,Mars.0,Mars.1,Mars.2,Mars.3,Mars.4,Mars.5,Earth.0,Earth.1,Earth.2,Earth.3,Earth.4,Earth.5
0,Diameter:,Mass:,Moons:,Distance from Sun:,Length of Year:,Temperature:,"6,779 km",6.39 × 10^23 kg,2,"227,943,824 km",687 Earth days,-87 to -5 °C,"12,742 km",5.97 × 10^24 kg,1,"149,598,262 km",365.24 days,-88 to 58°C


In [6]:
path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **path, headless = False)
url_images_2 = 'https://marshemispheres.com/'
browser.visit(url_images_2)
html = browser.html
soup = bs(html, 'html.parser')

h3 = soup.find_all('h3')[0:-1]


images_hemisphere = []



for i in h3:
    titles = i.text
    browser.links.find_by_partial_text(titles).click()
    html_en = browser.html
    soup_en = bs(html_en, 'html.parser')
    title = soup_en.find('h2', class_='title').text.replace('Enhanced', '').strip()
    sample = soup_en.find('li')
    imgurl = sample.find('a')['href']
    imgurl = url + imgurl

    hemi_dict = {'title': title, 'img_url': imgurl}
    images_hemisphere.append(hemi_dict)
    browser.links.find_by_partial_text('Back').click()

images_hemisphere



# html = browser.html
# soup = bs(html, 'html.parser')
# pic_link = bs(requests.get("https://marshemispheres.com/").text, "html.parser")
# # small_images = soup.find('a', class_ = 'itemlink product-item')

# image_list = []
# for a in list(pic_link):
#     f = soup.find('a', class_='itemLink product-item')
    
# f
# def init_browser():
#     executable_path = {"executable_path": "chromedriver.exe"}
#     return Browser("chrome", **executable_path, headless=False)

# browser = init_browser()
# url_images2 = 'https://marshemispheres.com/'
# browser.visit(url_images2)

# # browser.links.find_by_partial_text("Hemisphere Enhanced")
# listt = []
# b = browser.links.find_by_partial_text('Hemisphere Enhanced')
# b = (list(b))
# listt = listt.append(b)
# print(listt)
# for i in list(b):
#     browser.click_link_by_partial_text('Hemisphere Enhanced')

# browser.click_link_by_partial_text('Sample')
# # more_info_element.click()

# html = browser.html

# image_soup = bs(html)
# image_url = 



[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://redplanetscience.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://redplanetscience.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://redplanetscience.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://redplanetscience.com/images/valles_marineris_enhanced-full.jpg'}]

In [39]:

df = {
    'news_title': title,
    'news_paragraph': paragraph,
    'featured_image': imgurl,
    'facts': df_facts.to_json(),
    'hemiospheres': images_hemisphere
}




# df.to_excel('test.xlsx')


{'news_title': 'Valles Marineris Hemisphere',
 'news_paragraph': 'Two astronauts collected Moon rocks on Apollo 11. It will take three robotic systems working together to gather up the first Mars rock samples for return to Earth.',
 'featured_image': 'https://redplanetscience.com/images/valles_marineris_enhanced-full.jpg',
 'facts': '{"Mars - Earth Comparison":{"0":"Diameter:","1":"Mass:","2":"Moons:","3":"Distance from Sun:","4":"Length of Year:","5":"Temperature:"},"Mars":{"0":"6,779 km","1":"6.39 \\u00d7 10^23 kg","2":"2","3":"227,943,824 km","4":"687 Earth days","5":"-87 to -5 \\u00b0C"},"Earth":{"0":"12,742 km","1":"5.97 \\u00d7 10^24 kg","2":"1","3":"149,598,262 km","4":"365.24 days","5":"-88 to 58\\u00b0C"}}',
 'hemiospheres': [{'title': 'Cerberus Hemisphere',
   'img_url': 'https://redplanetscience.com/images/full.jpg'},
  {'title': 'Schiaparelli Hemisphere',
   'img_url': 'https://redplanetscience.com/images/schiaparelli_enhanced-full.jpg'},
  {'title': 'Syrtis Major Hemispher

In [7]:
browser.quit()